In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from random import choice

# constants mapping to index values in the data frame
id_index = 0
x_cord_index = 1
y_cord_index = 2
money_index = 3
time_index = 4
ratio_index = 5
value_index = 6
speed = 30

df = pd.read_csv('bank_data.csv')

# Adding a column to represent the attractiveness of banks 
df['ratio'] = df['money'] / df['time (hr)']

# numpy array of indexed ratios
ratio = df['ratio'].to_numpy()

# data frame as an np array for faster manipulations
data = df.to_numpy()

# Indexed array of banks and their time to escape point ex: times_to_escape[bank_id] -> 0.10315893
coords = df[df.columns.difference(['id', 'money', 'time (hr)', 'ratio'])].to_numpy()
times_to_escape = np.apply_along_axis(lambda a : distance.euclidean(a, [0,0]) / speed, 1, coords)

# matrix of each bank and the distance to other banks (indexed by id along both axes)
distances = distance.squareform(distance.pdist(coords))

closests = np.argsort(distance.squareform(distance.pdist(coords)), axis=1)

# matrix of each bank and the time to other banks (indexed by id along both axes)
times_to_point = distances / speed

In [2]:
from copy import deepcopy

def create_state_dict(visited, score, game_time):
    """
    takes all elements of a state
    returns a dictionary representation of that state
    """
    return {
            'visited' : visited,
            'score' : score,
            'game_time' : game_time,
        }

def get_ids_from_data(banks):
    """
    takes a list of visited bank information
    returns a lit of bank ids
    """
    return list(map(lambda x: x[id_index], banks))

class Board(object):
    def start(self):
        """
         Returns a representation of the starting state of a simulation(simulation are ran backwards)
         """
        return create_state_dict([], 0, 0)

    def play_move(self, state, play):
        """
        Takes the game state, and the move to be applied.
        Returns the new game state.
        """
        play = int(play)
        # On start we are at escape point. all
        if(len(state['visited']) == 0):
            return create_state_dict([play], data[play][money_index], data[play][time_index] + times_to_escape[play])
        
        distance_to_play = distance.euclidean(data[state['visited'][-1]][1:3], data[play][1:3])
        visited = [*state['visited'], play]
        score = state['score'] + data[play][money_index]
        game_time = state['game_time'] + data[play][time_index] + (distance_to_play / speed)

        return create_state_dict(visited, score, game_time)

    def is_play_legal(self, state, play):
        return play in state['visited']
        

    def legal_plays(self, state):
        """
        Takes a game state representing the game
        history, and returns the full list of moves that are legal plays
        """
        # if game is in starting state all moves are legal
        if(len(state['visited']) is 0):
            return data
        
        times_to_bank_copy = deepcopy(times_to_point[state['visited'][-1]])

         
        # any illegal moves becomes 0
        times_to_bank_copy[state['game_time'] + data[:, time_index] + times_to_bank_copy > 24] = 0

        for visit in state['visited']:
            times_to_bank_copy[visit] = 0

        # keep the indexes of legal moves        
        legal_moves = np.nonzero(times_to_bank_copy)[0]

        non_zero_legal_moves = np.take(data, legal_moves, axis=0)

        
        if(len(non_zero_legal_moves) > 0):
            return non_zero_legal_moves
        return []

    def winner(self, state):
        """
        takes the state_history
        returns a boolean indicating if the game is over
        """
    
        return len(self.legal_plays(state)) is 0

In [23]:
from check_solution import check_solution

original_df = pd.read_csv('bank_data.csv')

class Bot():
    def __init__(self, state={'visited': [], 'score': 0, 'game_time': 0}, radius=450):
        self.board = Board()
        self.state = state
        self.radius = radius
        self.game_over = self.board.winner(self.state)

    def pick_best_closest_move(self):
        legal_moves = self.board.legal_plays(self.state)
        if(len(legal_moves) < 10):
            return legal_moves[np.argsort(legal_moves[:, ratio_index])][-1]
        # 10000 is the 0,0 point
        closests_banks = closests[10000 if len(self.state['visited']) is 0 else self.state['visited'][-1]][0:self.radius]
        closest_legal_banks = legal_moves[np.in1d(legal_moves[:,0],closests_banks)]
        sorted_array = closest_legal_banks[np.argsort(closest_legal_banks[:, ratio_index])]
        
        if(np.isnan(sorted_array[-1][ratio_index])):
            return sorted_array[-2]
        else:
            return sorted_array[-1]

            # for bank in closest_legal_banks:
            #     state_copy = self.board.play_move(self.state, bank[id_index])
            #     second_legal_moves = self.board.legal_plays(state_copy)
            #     second_closests_banks = closests[int(bank[id_index])][0:99]
            #     second_closest_legal_banks = legal_moves[np.in1d(legal_moves[:,0],closests_banks)]

            #     sorted_array = second_closest_legal_banks[np.argsort(second_closest_legal_banks[:, ratio_index])]
                
            # print(sorted_array)
        

# for range 
board = Board()    
bot = Bot()
while(bot.game_over is False):
    bot.state = board.play_move(bot.state, bot.pick_best_closest_move()[id_index])
    bot.game_over = board.winner(bot.state)

print(bot.state)
check_solution(travel_list=np.flip(bot.state['visited']), df=original_df)

SyntaxError: invalid syntax (<ipython-input-23-673ce04dd518>, line 37)

False